In [1]:
!pip install spacy tabula https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.5.0/en_core_web_sm-3.5.0-py3-none-any.whl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 13.0 MB/s eta 0:00:00a 0:00:01


In [1]:
# Packages
import os
import re
import pdfplumber
import numpy
import pandas as pd
import spacy
from datetime import datetime

In [2]:
# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

# Load the provider info CSV for ending keywords
provider_info = pd.read_csv('provider.csv')

# Load the company info CSV for ticker validation and company metadata
company_info = pd.read_csv('company_info.csv')  # Replace with the actual path

company_info = company_info.drop_duplicates(subset='Ticker Symbol')

# Create a dictionary to map ticker symbols to company name and industry
ticker_map = company_info.set_index('Ticker Symbol')[['Company Name', 'Industry']].to_dict(orient='index')

In [3]:
# Only relevant to check font type and size for
import pdfplumber

def extract_words_with_formatting(page):
    """
    Extracts words along with their formatting details such as font size and font name.

    Args:
        page (pdfplumber.Page): A single page from the PDF.

    Returns:
        List[Dict]: A list of dictionaries containing words and their formatting details.
    """
    # Extract words with their bounding boxes
    words = page.extract_words(extra_attrs=["fontname", "size"])

    formatted_words = []
    for word in words:
        formatted_words.append({
            "word": word["text"],
            "font": word.get("fontname", "Unknown"),
            "size": word.get("size", "Unknown"),
            "x0": word["x0"],
            "x1": word["x1"],
            "top": word["top"],
            "bottom": word["bottom"]
        })
    return formatted_words


# Example usage with pdfplumber
pdf_path = "/Users/oskarroeske/Masterthesis/preprocessing/testing/20180124_Hilliard_Lyons_VZ_Report_received_in_email..pdf"

with pdfplumber.open(pdf_path) as pdf:
    for page_number, page in enumerate(pdf.pages, start=1):
        print(f"Page {page_number}:")
        formatted_words = extract_words_with_formatting(page)
        for word_info in formatted_words:
            print(
                f"Word: '{word_info['word']}', Font: {word_info['font']}, Size: {word_info['size']}, "
                f"Position: ({word_info['x0']}, {word_info['top']} - {word_info['x1']}, {word_info['bottom']})"
            )


Page 1:
Word: 'p', Font: TimesNewRomanPSMT, Size: 12.0, Position: (72.0, 225.43199999999945 - 78.0, 237.43199999999945)
Word: 'Note', Font: TimesNewRomanPS-BoldMT, Size: 15.0, Position: (170.16, 664.4999 - 200.1315, 679.4999)
Word: 'Important', Font: TimesNewRomanPS-BoldMT, Size: 15.0, Position: (203.90699999999998, 664.4999 - 270.53099999999995, 679.4999)
Word: 'Disclosures', Font: TimesNewRomanPS-BoldMT, Size: 15.0, Position: (274.30649999999997, 664.4999 - 346.7565000000001, 679.4999)
Word: 'on', Font: TimesNewRomanPS-BoldMT, Size: 15.0, Position: (350.53200000000015, 664.4999 - 366.3675000000001, 679.4999)
Word: 'Pages', Font: TimesNewRomanPS-BoldMT, Size: 15.0, Position: (370.14300000000014, 664.4999 - 406.78500000000014, 679.4999)
Word: '2', Font: TimesNewRomanPS-BoldMT, Size: 15.0, Position: (410.5605000000003, 664.4999 - 418.0605000000003, 679.4999)
Word: '-', Font: TimesNewRomanPS-BoldMT, Size: 15.0, Position: (421.83600000000024, 664.4999 - 426.83100000000024, 679.4999)
Word:

In [34]:
# Test new Structure

patterns = {
    "BGC Partners": {
        "price_patterns": {
            "primary": r"Price Target \(\$\) (\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"([A-Za-z]+) \(\w+,",
        },
        "ending_patterns": [r"Disclosures Appendix"],
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Needham": {
        "price_patterns": {
            "primary": r"Price Target: \$(\d{1,3}(,\d{3})*(\.\d{2})?)",
            "secondary": r"PRICE TARGET: \$(\d{1,3}(,\d{3})*(\.\d{2})?)",
        },
        "rating_patterns": {
            "primary": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
            "secondary": r"Rating (\w+)",
        },
        "ending_patterns": [r"Analyst Certification"],
        "font_patterns": [
            {"font_type": r"E[A-Z]+[+]Cambria(-Bold)?", "font_size": 10.0},
        ]
    },
    "BTIG": {
        "price_patterns": {
            "primary": r"\$(\d+(\.\d+)?) 12 month target ",
        },
        "rating_patterns": {
            "primary": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
        },
        "ending_patterns": [r"Appendix: Analyst Certification and Other Important Disclosures"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Corbel(,Bold)?(,-Italic)?", "font_size": 9.96},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Wells Fargo": {
        "price_patterns": {
            "primary": r"\/Price Target: \$(\d+(\.\d+)?)",
            "secondary": r"Price Target\/Prior: \$(\d+(\.\d+)?)",
            "tertiary": r"\/\$(\d+(\.\d+)?)"
        },
        "rating_patterns": {
            "primary": r"([A-Za-z]+)/\$",
            "secondary": r"Rating (\w+)",
        },
        "ending_patterns": [r"Required Disclosures"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]WellsFargoSans(-Light)?(,-SemiBold)?", "font_size": 9.00},
            {"font_type": r"[A-Z]+[+]Verdana(-Bold)?", "font_size": 8.04},
            {"font_type": r"[A-Z]+[+]DejaVuSans(-Bold)?", "font_size": 9.01}
        ]
    },
    "Barclays": {
        "price_patterns": {
            "primary": r"Price Target USD (\d+(\.\d+)?)",
            "secondary": r"Price Target: USD (\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Stock Rating ([A-Za-z]+)",
        },
        "ending_patterns": [r"ANALYST\(S\) CERTIFICATION\(S\)",
                            r"Analyst\(s\) Certification\(s\)"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Expert Sans (Extra Bold)?(Regular)?(Regular,Bold)?", "font_size": 9.0},
            {"font_type": r"[A-Z]+[+]Expert Sans (Extra Bold)?(Regular)?", "font_size": 8.04},
            {"font_type": r"[A-Z]+[+]DejaVuSans(-Bold)?", "font_size": 9.01},
        ]
    },
    "JP Morgan": {
        "price_patterns": {
            "primary": r"Price Target \([A-Za-z0-9\-]+\): \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"\b(Buy|Hold|Sell|Overweight|Underperform)\b",
        },
        "ending_patterns": [r"Analyst Certification"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]TimesNewRoman(,Bold)?", "font_size": 9.60},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Brean Capital LLC": {
        "price_patterns": {
            "primary": r"PT: \$ (\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"([A-Za-z]+) PT:\$",
        },
        "ending_patterns": [r"Analyst Certification"],
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Hilliard Lyons": {
        "price_patterns": {
            "primary": r"Price Target (NA|\$(\d+(\.\d+)?))",
        },
        "rating_patterns": {
            "primary": r"-- NYSE\s+[–\-—]+\s+([A-Za-z]+)\s+[–\-—]+",
            "secondary":r"NYSE\s+[–\-—]+\s+([A-Za-z\- ]+?)(?=\s*[-–—]\d)"
        },
        "ending_patterns": [r"Analyst Certification"],
        "font_patterns": [
            {"font_type": r"Verdana(-Bold)?", "font_size": 9.00},
            {"font_type":r"TimesNewRomanPS(-BoldMT)?", "font_size": 10.98},
            ]
    },
    "Alliance Global Partners": {
        "price_patterns": {
            "primary": r"Price Target (NA|\$(\d+(\.\d+)?))",
        },
        "rating_patterns": {
            "primary": r"\b(Buy|Hold|Sell|Overweight|Underperform)\b",
        },
        "ending_patterns": [r"Imporant Research Disclosures"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]ArialMT(-BoldMT)?", "font_size": 8.00},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Mizuho Securities": {
        "price_patterns": {
            "primary": r"Price Target \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Rating ([A-Za-z]+)",
        },
        "ending_patterns": [r"IMPORTANT DISCLOSURES"],
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Gilford Securities Inc": {
        "price_patterns": {
            "primary": r"\, \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Rated: ([A-Za-z]+)",
            "secondary": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
        },
        "ending_patterns": r"ANALYST CERTIFICATION",
        "font_patterns": [
            {"font_type": r"ArialMT(-BoldMT)?", "font_size": 10.02},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Deutsche Bank": {
        "price_patterns": {
            "primary": r"Price Target \(USD\) (\d+(\.\d+)?)",
            "secondary": r"Price target (\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Rating ([A-Za-z]+)",
            "secondary": r"\b(Buy|Hold|Sell|Overweight|Underperform)\b",
        },
        "ending_patterns": [r"Appendix 1"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]UniversDeutscheBank-Regular", "font_size": 9.0},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Pivotal Research Group": {
        "price_patterns": {
            "primary": r"Target Price: \$(\d+(,\d+)?)",
        },
        "rating_patterns": {
            "primary": r"RATING: ([A-Za-z]+)",
        },
        "ending_patterns": [r"Appendix: Important Disclosures"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Helvetica(-Bold)?", "font_size": 9.96},
            {"font_type": r"[A-Z]+[+]Arial", "font_size": 8.04},
        ]
    },
    "Spartan Capital Securities LLC": {
        "price_patterns": {
            "primary": r"T \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"([A-Za-z]+)",
        },
        "ending_patterns": [r"Important Disclosures"],
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Cascend Securities -Historical-": {
        "price_patterns": {
            "primary": r"Price target: \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Rating: ([A-Za-z]+)",
        },
        "ending_patterns": [r"Disclosures: "],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Calibri(,Bold)?", "font_size": 12.0},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Phillip Securities": {
        "price_patterns": {
            "primary": r"TARGET PRICE USD (\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
        },
        "ending_patterns": [r"Contact Information"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Calibri(-Bold)?", "font_size": 9.96},
            {"font_type": r"[A-Z]+[+]Calibri(-Bold)?", "font_size": 10.0},
        ]
    },
    "FinTrust Investment Advisors": {
        "price_patterns": {
            "primary": r"Target Price: \$(\d+(,\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Fintrust Rating: ([A-Za-z]+)",
        },
        "ending_patterns": [r"Important Disclosures:"],
        "font_patterns": [
            {"font_type": r"Arial(-BoldMT)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "IBI Investment House": {
        "price_patterns": {
            "primary": r"Price target: \$(\d+(,\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Recommendation: ([A-Za-z]+)",
        },
        "ending_patterns": [r"Disclosures"],
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    }
}



In [35]:
def check_validity(paragraph):
    # Parse the paragraph
    paragraph = re.sub(r"\s+", " ", paragraph).strip()
    doc = nlp(paragraph)
        
    for sent in doc.sents:
        has_verb = False
        has_subject = False
        
        for token in sent:
            # Check for a verb
            if token.pos_ in {"VERB", "AUX"}:
                has_verb = True
            # Check for a subject
            if token.dep_ in {"nsubj", "nsubjpass"}:
                has_subject = True
        
        # If both a verb and a subject are found, the sentence is valid
        if has_verb and has_subject:
            return True
        
        # At least one word with 5+ letters and all upper case
        if re.search(r"[A-Z]{5,}", paragraph):
            return True
    return False

def filter_valid_paragraphs(paragraphs):
    valid_paragraphs = []

    for paragraph in paragraphs:
        if check_validity(paragraph):  # Validate each paragraph
            valid_paragraphs.append(paragraph)
    return valid_paragraphs

def extract_text_with_format(page, provider):
    print(f"Provider: {provider}")

    provider_patterns = patterns[provider]

    # Access patterns
    price_patterns = provider_patterns["price_patterns"]
    rating_patterns = provider_patterns["rating_patterns"]
    ending_patterns = provider_patterns["ending_patterns"]
    font_patterns = provider_patterns["font_patterns"]

    # Extract words with font and size details
    words = page.extract_words(extra_attrs=["fontname", "size"])

    # Round text sizes to 3 decimal places, moved it to lower part, hope this is correct
    """for word in words:
        if "size" in word and word["size"] is not None:
            word["size"] = round(word["size"], 2)"""

    # Sort words by vertical and horizontal position
    words.sort(key=lambda w: (w["top"], w["x0"]))

    paragraphs = []
    current_paragraph = []
    current_top = None

    rating = None
    price = None

    # Lookahead buffer for multi-word patterns
    lookahead_buffer = []

    for word in words:

        if "size" in word and word["size"] is not None:
            word["size"] = round(word["size"], 2)

        # Build lookahead buffer
        lookahead_buffer.append(word["text"])
        if len(lookahead_buffer) > 10:
            lookahead_buffer.pop(0)
        buffer_text = " ".join(lookahead_buffer)

        print(buffer_text)

        # Extract rating
        if not rating:
            for pattern_key in ["primary", "secondary","tertiary"]:
                pattern = rating_patterns.get(pattern_key)
                if pattern:
                    rating_match = re.search(pattern, buffer_text)
                    if rating_match:
                        rating = rating_match.group(1)
                        break

        # Extract price
        if not price:
            for pattern_key in ["primary", "secondary"]:
                pattern = price_patterns.get(pattern_key)
                if pattern:
                    price_match = re.search(pattern, buffer_text)
                    if price_match:
                        price = price_match.group(1)
                        break

        # Check for ending pattern
        # Check if buffer matches any ending pattern
        for ending_pattern in ending_patterns:
            if re.search(ending_pattern, buffer_text):
                return filter_valid_paragraphs(paragraphs), True, rating, price

        # Match word against font patterns
        is_font_matched = False
        for font_pattern in font_patterns:
            font_type = font_pattern["font_type"]
            font_size = font_pattern["font_size"]
            if re.match(font_type, word["fontname"]) and word["size"] == font_size:
                is_font_matched = True
                #break

        if not is_font_matched:
            continue

        # Group words into paragraphs
        if current_top is None or abs(word["top"] - current_top) < 13:  # Adjust threshold as needed
            current_paragraph.append(word["text"])
        else:
            # New paragraph starts
            paragraphs.append(" ".join(current_paragraph))
            current_paragraph = [word["text"]]

        # Update the current top position
        current_top = word["top"]

    # Add the last paragraph
    if current_paragraph:
        paragraphs.append(" ".join(current_paragraph))

    return filter_valid_paragraphs(paragraphs), False, rating, price


# NEW VERSION (16.11.2024)
def extract_metadata(filename, ticker_map):
    """
    Extract metadata (date, provider, ticker) from the filename using ticker_map.
    """
    # Extract the date (first 8 digits in the filename)
    date_match = re.match(r"(\d{8})", filename)
    if not date_match:
        return None, None, None, None, None
    date_str = date_match.group(1)
    date = datetime.strptime(date_str, "%Y%m%d")

    print(f"fileName: {filename}")
    print(f"date: {date}")

    # Look for the ticker in the filename
    for ticker in ticker_map.keys():
        ticker_pattern = f"_{ticker}_"  # Ensure ticker is surrounded by underscores
        if ticker_pattern in filename:
            # Extract the portion between date and ticker as the provider
            provider_section = filename.split(f"{date_str}_")[1].split(f"_{ticker}_")[0]
            provider = provider_section.replace('_', ' ')  # Replace underscores with spaces
            # Get company name and industry from the ticker_map
            company_name = ticker_map[ticker]['Company Name']
            industry = ticker_map[ticker]['Industry']
            return date, provider, ticker, company_name, industry

    # If no ticker is found, return None for ticker-related fields
    return date, None, None, None, None


In [36]:

# Define function to process all PDFs in the directory and store data in DataFrame
def extract_text_from_all_pdfs_to_dataframe(directory_path, provider_info, ticker_map):
    data = []
    id_counter = 1  # Initialize an ID counter
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):  # Process only PDF files
            file_path = os.path.join(directory_path, filename)

            
            # Extract metadata from filename
            date, provider, ticker, company_name, industry = extract_metadata(filename,ticker_map=ticker_map)

            all_paragraphs = []  
            first_rating = None
            first_price = None  
            stop_extraction = False

            with pdfplumber.open(file_path) as pdf:
                for page_number, page in enumerate(pdf.pages, start=1):
                    if stop_extraction:
                        break  # Exit the loop if stop_extraction is set
            
                    # Extract data from the current page
                    paragraphs, stop_extraction, rating, price = extract_text_with_format(page, provider=provider)
                    
                    # Append paragraphs from the current page
                    all_paragraphs.extend(paragraphs)

                    # Capture the first non-None rating and price
                    if rating is not None and first_rating is None:
                        first_rating = rating
                    if price is not None and first_price is None:
                        first_price = price
                        
            # Add extracted data to the list
            data.append({
                "ID": id_counter,  # Unique ID
                "filename": filename,
                "date": date,
                "provider": provider,
                "ticker": ticker,
                "company_name": company_name,
                "industry": industry,
                "paragraphs": all_paragraphs,
                "target_price":first_price,
                "rating": first_rating.lower() if first_rating else None
                })
            id_counter += 1  # Increment the ID counter for the next row

    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df

# Directory path
pdf_directory = "../preprocessing/testing"  # Replace with your actual folder path

# Run the function and store results in a DataFrame
df_reports = extract_text_from_all_pdfs_to_dataframe(pdf_directory, provider_info, ticker_map)

# Display the resulting DataFrame to confirm
df_reports.head(15)

fileName: 20170728_Hilliard_Lyons_VZ_Report_received_in_email..pdf
date: 2017-07-28 00:00:00
Provider: Hilliard Lyons
Telecommunications
Telecommunications Analyst:
Telecommunications Analyst: David
Telecommunications Analyst: David Burks
Telecommunications Analyst: David Burks 502.588.8648
Telecommunications Analyst: David Burks 502.588.8648 /
Telecommunications Analyst: David Burks 502.588.8648 / DBurks@hilliard.com
Telecommunications Analyst: David Burks 502.588.8648 / DBurks@hilliard.com Institutional
Telecommunications Analyst: David Burks 502.588.8648 / DBurks@hilliard.com Institutional Sales
Telecommunications Analyst: David Burks 502.588.8648 / DBurks@hilliard.com Institutional Sales Desk:
Analyst: David Burks 502.588.8648 / DBurks@hilliard.com Institutional Sales Desk: George
David Burks 502.588.8648 / DBurks@hilliard.com Institutional Sales Desk: George Moorin
Burks 502.588.8648 / DBurks@hilliard.com Institutional Sales Desk: George Moorin 502.588.9141
502.588.8648 / DBurks@h

,ID,filename,date,provider,ticker,company_name,industry,paragraphs,target_price,rating
0,1,20170728_Hilliard_Lyons_VZ_Report_received_in_...,2017-07-28,Hilliard Lyons,VZ,Verizon Communications Inc.,Communication Services,[Verizon reported second quarter adjusted earn...,NA,neutral
1,2,20180426_Hilliard_Lyons_VZ_.pdf,2018-04-26,Hilliard Lyons,VZ,Verizon Communications Inc.,Communication Services,[Company reports solid first quarter results; ...,NA,neutral
2,3,20170731_Hilliard_Lyons_XOM_Report_received_in...,2017-07-31,Hilliard Lyons,XOM,Exxon Mobil Corp.,Energy,[XOM reported Q2’17 EPS of $0.78 vs. ($0.41) a...,$90.00,long-term buy
3,4,20180124_Hilliard_Lyons_VZ_Report_received_in_...,2018-01-24,Hilliard Lyons,VZ,Verizon Communications Inc.,Communication Services,[Verizon reported fourth quarter adjusted earn...,NA,neutral


# Cleaning of paragraphs

In [324]:
import re

def clean_paragraph(paragraph):
    # Remove email addresses
    paragraph = re.sub(r"\S+@\S+", "", paragraph)
    
    # Remove phone numbers
    paragraph = re.sub(r"\b\d{1,3}[-.\s]?\d{3}[-.\s]?\d{4}\b", "", paragraph)
    
    # Remove URLs
    paragraph = re.sub(r"http\S+|www\S+", "", paragraph)
    
    # Remove special characters (keep alphanumerics, spaces, and common punctuation)
    paragraph = re.sub(r"[^\w\s,.!?]", "", paragraph)
    
    # Remove multiple spaces or newlines
    paragraph = re.sub(r"\s+", " ", paragraph).strip()
    
    # Remove boilerplate phrases
    boilerplate_phrases = ["Disclaimer", "Confidential", "For internal use only"]
    for phrase in boilerplate_phrases:
        paragraph = paragraph.replace(phrase, "")
    
    # Remove short sentences (fewer than 10 words)
    if len(paragraph.split()) < 6:
        return None
    
    return paragraph

# Apply cleaning function
df_reports['paragraphs'] = df_reports['paragraphs'].apply(
    lambda paragraphs: [clean_paragraph(p) for p in paragraphs if clean_paragraph(p)]
)

# Drop rows where the `paragraphs` column is empty after cleaning
df_reports = df_reports[df_reports['paragraphs'].str.len() > 0]

df_reports["rating"] = df_reports["rating"].astype(str).apply(lambda x: x.lower() if x != "nan" else None)

# Get Date and Price for Dates

In [325]:
df_reports.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26 entries, 0 to 26
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ID            26 non-null     int64         
 1   filename      26 non-null     object        
 2   date          26 non-null     datetime64[ns]
 3   provider      26 non-null     object        
 4   ticker        26 non-null     object        
 5   company_name  26 non-null     object        
 6   industry      26 non-null     object        
 7   paragraphs    26 non-null     object        
 8   target_price  21 non-null     object        
 9   rating        26 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 2.2+ KB


In [327]:
# Load and ensure 'Date' is in datetime format
df_performance_data = pd.read_csv("performance_data.csv")

# Convert 'Date' column to datetime and set it as index (with timezone awareness)
df_performance_data['Date'] = pd.to_datetime(df_performance_data['Date'], utc=True)
df_performance_data = df_performance_data.set_index('Date')

In [328]:
df_saved_reports = df_reports

In [329]:
def get_stock_prices(ticker, start_date, end_date=None):
    try:
        # Ensure start_date and end_date are in the same timezone (UTC)
        start_date = pd.to_datetime(start_date, utc=True)
        if end_date is not None:
            end_date = pd.to_datetime(end_date, utc=True)
        else:
            end_date = start_date  # If no end_date, use start_date for a single day

        # Filter the data for the ticker and date range
        filtered_data = df_performance_data.loc[
            (df_performance_data.index >= start_date) & 
            (df_performance_data.index <= end_date), ticker]

        if not filtered_data.empty:
            # Return the maximum price within the filtered date range
            max_price = filtered_data.max()
            min_price = filtered_data.min()
            return float(max_price), float(min_price)
        else:
            # If no data available in the range, find the next available date using asof()
            next_available_data = df_performance_data[ticker].asof(start_date)
            if next_available_data is not None:
                return float(next_available_data), float(next_available_data)
            else:
                return float('nan'), float('nan')  # Return NaN for missing data

    except KeyError:
        return float('nan'), float('nan')  # Return NaN for missing data
    except Exception as e:
        return float('nan'), float('nan')  # Return NaN for missing data

def calculate_prices(row):
    short_name = row['ticker']
    base_date = pd.to_datetime(row['date'], utc=True)

    # Calculate prices
    row['start price'] = get_stock_prices(short_name, base_date)[0]
    row['one day after'] = get_stock_prices(short_name, base_date + pd.DateOffset(days=1))[0]
    row['max price after 3 months'], row["min price after 3 months"] = get_stock_prices(
        short_name, base_date, base_date + pd.DateOffset(months=3)
    )
    row['max price after 6 months'], row['min price after 6 months'] = get_stock_prices(
        short_name, base_date + pd.DateOffset(months=3), base_date + pd.DateOffset(months=6)
    )
    row['max price after 9 months'], row['min price after 9 months'] = get_stock_prices(
        short_name, base_date + pd.DateOffset(months=6), base_date + pd.DateOffset(months=9)
    )
    row['max price after 12 months'], row['min price after 12 months'] = get_stock_prices(
        short_name, base_date + pd.DateOffset(months=9), base_date + pd.DateOffset(months=12)
    )
    return row

# Apply the function to each row
df_saved_reports = df_saved_reports.apply(calculate_prices, axis=1)


In [330]:
df_saved_reports.head(10)

,ID,filename,date,provider,ticker,company_name,industry,paragraphs,target_price,rating,start price,one day after,max price after 3 months,min price after 3 months,max price after 6 months,min price after 6 months,max price after 9 months,min price after 9 months,max price after 12 months,min price after 12 months
0,1,20211105_Wells_Fargo_XOM_XOM-_Post_Q3-21_Resul...,2021-11-05,Wells Fargo,XOM,Exxon Mobil Corp.,Energy,[We maintain our Overweight rating and raise o...,75.00,overweight,57.704025,57.704025,73.220833,53.208992,83.390045,68.930939,96.086212,76.380234,104.181046,77.901566
1,2,20170523_BTIG_CRM_Salesforce.com-_Inc..pdf,2017-05-23,BTIG,CRM,Salesforce.com Inc.,Technology,[Salesforce reported excellent F1Q17 results a...,100.00,buy,88.098900,89.203995,92.658653,85.619904,108.319130,91.931877,114.451904,99.408684,130.052658,112.381096
2,3,20210820_Phillip_Securities_AAPL_Apple_Inc_Sup...,2021-08-20,Phillip Securities,AAPL,Apple Inc.,Technology,[Company Background Apples revenue comes from ...,None,buy,145.589172,145.589172,157.962463,136.698029,179.076599,154.282730,176.300095,135.507355,172.447479,128.315125
3,4,20201201_Barclays_MRK_MRK-_Thoughts_on_OncoImm...,2020-12-01,Barclays,MRK,Merck & Co.,Healthcare,[Merck acquired OncoImmune Private for 425mn t...,90.00,none,68.896400,69.132957,72.378143,61.632141,68.607460,61.453327,71.574394,64.578827,82.943298,65.665733
4,5,20220209_JP_Morgan_PEP_PepsiCo-_4Q_Preview-_Ex...,2022-02-09,JP Morgan,PEP,PepsiCo Inc.,Consumer Staples,"[PepsiCo PEP, Overweight rated is scheduled to...",185.00,overweight,159.078400,155.775436,165.302597,143.166016,165.936584,146.403046,172.036377,152.569839,176.595840,166.844055
5,6,20220202_Wells_Fargo_T_T-_Holding_Pattern_on_U...,2022-02-02,Wells Fargo,T,AT&T Inc.,Communication Services,[While we thought nancial and operational Q4 r...,26.00,equal,15.633189,15.645931,17.291956,14.582060,18.241688,15.866283,16.306925,12.909647,17.030497,15.759830
6,7,20201105_Wells_Fargo_V_V-_Cross_Border_Earning...,2020-11-05,Wells Fargo,V,Visa Inc.,Financials,"[Summary V reported FYQ4 adj EPS of 1.12, 5c a...",210,overweight,191.666595,192.471481,212.438782,187.691666,230.404480,199.150146,244.445541,214.616791,235.161804,202.998688
7,8,20140203_BGC_Partners_AAPL_BGC_AAPL_01272014.pdf,2014-02-03,BGC Partners,AAPL,Apple Inc.,Technology,[APPLE EARNINGS PREVIEW DOWNGRADE TO HOLD AS S...,550,hold,15.626557,15.852774,18.621323,15.626557,21.847317,18.455833,24.257574,20.949322,26.500629,23.603365
8,9,20140808_Needham_T_TWX-_2Q14_Preview-_Maintain...,2014-08-08,Needham,T,AT&T Inc.,Communication Services,[INVESTMENT HIGHLIGHTS NA We make no changes t...,None,hold,12.182156,12.182156,12.640379,12.097363,12.910082,11.557942,13.016914,11.951909,13.509806,12.505348
10,11,20200205_Barclays_VZ_VZ-_Margin_pressure_conti...,2020-02-05,Barclays,VZ,Verizon Communications Inc.,Communication Services,[Verizon was the fourth straight carrier to be...,55.00,none,45.343903,45.581623,46.195099,38.296543,45.390003,41.204220,47.506271,44.605843,48.907204,43.458092


In [331]:
df_saved_reports["target_price"] = (
    df_saved_reports["target_price"]
    #Remove comma and transform to float for later operations
    .replace(",", "", regex=True) 
    .astype(float)                 
)

# Calculate performance of Target Prices

In [332]:
#If the target_price is below the current price, we need to check if it was below the target_price
def classifcy_performance(df):
    if df["target_price"] > df["start price"]:
        df["tp reached after 3 months"] = df["target_price"] <= df["max price after 3 months"]
        df["tp reached after 6 months"] = df["target_price"] <= df["max price after 6 months"]
        df["tp reached after 9 months"] = df["target_price"] <= df["max price after 9 months"]
        df["tp reached after 12 months"] = df["target_price"] <= df["max price after 12 months"]
    else:
        df["tp reached after 3 months"] = df["target_price"] >= df["min price after 3 months"]
        df["tp reached after 6 months"] = df["target_price"] >= df["min price after 6 months"]
        df["tp reached after 9 months"] = df["target_price"] >= df["min price after 9 months"]
        df["tp reached after 12 months"] = df["target_price"] <= df["min price after 12 months"]

    return df

df_saved_reports = df_saved_reports.apply(classifcy_performance, axis=1)

# Clean/Filter Paragraphs

In [333]:
df_saved_reports.value_counts("rating")

rating
overweight     7
buy            6
none           6
hold           2
unchanged      2
change         1
equal          1
underweight    1
Name: count, dtype: int64

In [334]:
df_saved_reports.to_csv("/Users/oskarroeske/Masterthesis/full_analysis/preprocessed_reports.csv")

In [335]:
# Explode the cleaned list into separate rows
df_exploded = df_saved_reports.explode('paragraphs').reset_index(drop=True)

In [336]:
df_exploded = df_exploded[["ID","paragraphs"]]

In [337]:
df_exploded.to_csv("/Users/oskarroeske/Masterthesis/full_analysis/preprocessed_paragraphs.csv")

In [338]:
#df_reports.to_csv("paragraphs.csv")